<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#En-X86." data-toc-modified-id="En-X86.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>En X86.</a></span></li><li><span><a href="#Trouver-la-faille-?" data-toc-modified-id="Trouver-la-faille-?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Trouver la faille ?</a></span></li><li><span><a href="#Aide" data-toc-modified-id="Aide-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Aide</a></span></li><li><span><a href="#Solution" data-toc-modified-id="Solution-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Solution</a></span></li><li><span><a href="#Comment-l'éviter-?" data-toc-modified-id="Comment-l'éviter-?-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Comment l'éviter ?</a></span></li></ul></div>

# Injection

Le cas le plus courant/connu d'injection est l'injection sql (voir https://fr.wikipedia.org/wiki/Injection_SQL). 

L' "injection" est une faille pouvant arriver quand un programme produit un programme ou des éléments +/- exécutables à partir de données saisies interactivement. (il y a donc 2 programmes en jeu, un programme principal/producteur et un programme produit, et une saisie interactive dans le programme principal/producteur donnant les éléments pour que ce programme construise le programme produit). 

Pour donner un exemple dans le contexte d'un cours de représentation de l'information, prenons un cas de programme C qui attend une chaine de caractères et produit avec cette entrée un programme qui vérifiera que cette chaine correspond bien à un nom connu (ici "John") et écrira "Bonjour" si c'est le cas (et rien ou "Pardon ?" sinon).  

Le programme produit sera donc proche de celui-là (la chaîne saisie sera à mettre à la place de A-REMPLIR par le programme principal/producteur) :

In [1]:
%%writefile bonjourInjection.c
#include <stdio.h>
#include <string.h>

int main () {
    if (strcmp("John","A-REMPLIR")==0) {
        printf("Bonjour\n");}
    else {printf("Pardon ?\n");}
    return 0;}

Writing bonjourInjection.c


Compilation de ce programme

In [2]:
%%sh
arm-linux-gnueabi-gcc -S -static bonjourInjection.c
cat bonjourInjection.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"bonjourInjection.c"
	.section	.rodata
	.align	2
.LC0:
	.ascii	"Pardon ?\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 0
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	ldr	r0, .L3
	bl	puts
	mov	r3, #0
	mov	r0, r3
	ldmfd	sp!, {fp, pc}
.L4:
	.align	2
.L3:
	.word	.LC0
	.size	main, .-main
	.ident	"GCC: (Ubuntu/Linaro 4.7.3-12ubuntu1) 4.7.3"
	.section	.note.GNU-stack,"",%progbits


Exécution de ce programme

In [6]:
%%sh
arm-linux-gnueabi-gcc -static bonjourInjection.c
qemu-arm a.out

Pardon ?


## En X86.

In [13]:
%%sh
gcc bonjourInjection.c
./a.out

Pardon ?


Le programme principal/producteur (celui qui produit le programme bonjour), prend donc un paramètre et produit le programme bonjourInjection en fonction (en remplaçant A-REMPLIR par le paramètre). Ce programme est donc le suivant :

In [7]:
%%writefile pourBonjourInjection.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

int main (int argc, char * argv[]) {
char command[1000];
    if (argc==1) {
        printf("Il faut un argument\n");}
    else {
        system("rm a.out");
        system("echo \"#include <stdio.h>\" > bonjourInjection.c");
        system("echo \"#include <string.h>\" >> bonjourInjection.c");
        system("echo \"int main () {\" >> bonjourInjection.c");
        strcpy(command,"echo \"if (strcmp(\\\"John\\\",\\\"");
        strcat(command,                                     argv[1]);
        strcat(command,                                             "\\\")==0) {\">> bonjourInjection.c");
        system(command);
        system("echo \"  printf(\\\"Bonjour\\\");}\">> bonjourInjection.c");
        system("echo \"else {printf(\\\"Pardon ?\\\");}\" >> bonjourInjection.c");
        system("echo \"return 0;}\" >> bonjourInjection.c");
        system("gcc bonjourInjection.c");
        system("./a.out");
    }
    return 0;}

Writing pourBonjourInjection.c


Le fonctionnement prévu est le suivant :
* pour John, il dit "Bonjour"
* pour Joe (ou tout le reste), il dit "Pardon ?"

In [8]:
%%sh
gcc -o pourBonjourInjection.e pourBonjourInjection.c
./pourBonjourInjection.e Joe

Pardon ?

In [12]:
%%sh
gcc -o pourBonjourInjection.e pourBonjourInjection.c
./pourBonjourInjection.e John

Bonjour

## Trouver la faille ?

Le jeu, maintenant, c'est de trouver comment lui faire dire bonjour, sans dire John.

In [13]:
%%sh
gcc -o pourBonjourInjection.e pourBonjourInjection.c
./pourBonjourInjection.e MettreCeQueVousVoulezMaisPasNImporteQuoi

Pardon ?

In [14]:
%%sh
cat bonjourInjection.c

#include <stdio.h>
#include <string.h>
int main () {
if (strcmp("John","MettreCeQueVousVoulezMaisPasNImporteQuoi")==0) {
  printf("Bonjour");}
else {printf("Pardon ?");}
return 0;}


## Aide 

Comment faire ?  

L'objectif, c'est de mettre une chaîne de caractères 
* qui ne provoque pas une erreur de compilation
* mais qui détourne le test d'égalité pour ne pas le faire vraiment
* qui remplace ce test non fait ou mal fait ou fait de manière erronée, en faisant comme s'il avait réussi,
* et qui poursuive le reste du code comme si tout était ok

Les moyens pour cela :
* ajouter des caractères spéciaux dans la chaîne de caractères
* en particulier des caractères de fin de chaîne
* mais en général, on ne peut pas en mettre ces caractères de fin de chaîne sans caractère spéciaux (dit caractères d'échapement)
* il faudra aussi mettre des morceaux de code C, éventuellement après la fin de chaîne, pour faire que le code compile, et s'exécute
* etc.



## Solution

La Solution ?  

Une solution ?

Vous voulez la solution ?

Vous voulez une solution ?

Toute la solution ?

Juste un morceau ?

Pour finir la chaîne :
* "

Dans un chaîne ?  

Pour pouvoir finir la chaîne :
* un échapement \

Mais pour avoir un échapement il faut être dans une chaîne ?

Et dans cette chaine, l'échapement, il échappe quoi ?

Bref ...

Dans la chaîne, il faudra mettre :
* "\\\" ....."

Ensuite, pour tromper le test :
* " ... )!=0 ...."

Puis pour éviter le reste de la ligne (et l'évaluation initiale du test :
* " ... { // ..."

Voila !


Au final :
* "\\\")!=0) { // "

In [10]:
%%sh
gcc -o pourBonjourInjection.e pourBonjourInjection.c
./pourBonjourInjection.e "\\\")!=0) { // "

Bonjour

In [11]:
%%sh
cat bonjourInjection.c

#include <stdio.h>
#include <string.h>
int main () {
if (strcmp("John","")!=0) { // ")==0) {
  printf("Bonjour");}
else {printf("Pardon ?");}
return 0;}


## Comment l'éviter ?

Il y a plusieurs choses à éviter, ou dont il faut se méfier :
* les codes produits à la volée
* les entrées/sorties
* les chaînes de caractères

Et si on ne peut pas éviter tout cela :
* vérifier qu'une chaîne de caractères est bien aussi simple que prévue 
* (ici, n'auroriser que les lettres a-zA-Z et vérifier que l'entrée est bien seulement constituée de a-ZA-Z)